In [ ]:
# Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.
# Create a Jupyter Notebook file called mission_to_mars.ipynb and use this to complete all of your scraping and analysis tasks. 
# The following outlines what you need to scrape.

In [39]:
#Import the dependancies 
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template
from pprint import pprint
import pymongo
import pandas as pd
import time
import os

In [40]:
#finding the chrome driver and telling it to use Chrome:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path)

#going to mars! ...well the news page at least
url = "https://mars.nasa.gov/news/"
browser.visit(url)

#using bs to write it into html
html = browser.html
soup = bs(html,"html.parser")

In [41]:
# NASA Mars News
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.
news_title = soup.find("div",class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text
print(f"Title: {news_title}")
print(f"Paragraph: {news_p}")

Title: For InSight, Dust Cleanings Will Yield New Science
Paragraph: Wind can be crucial to clearing dust from spacecraft solar panels on Mars. With InSight's meteorological sensors, scientists get their first measurements of wind and dust interacting "live" on the Martian surface.  


In [42]:
# JPL Mars Space Images - Featured Image
# Visit the url for JPL Featured Space Image here.
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
# and assign the url string to a variable called featured_image_url.
# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.

# # Example:
# featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
# print(featured_image_url)
# browser.visit(featured_image_url)

In [43]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

# featured_img_base = "https://www.jpl.nasa.gov"
# featured_img_url_raw = soup.find("div", class_="carousel_items").find("article")["style"]
# featured_img_url = featured_img_url_raw.split("'")[1]
# featured_img_url = featured_img_base + featured_img_url
# featured_img_url
# print(featured_image_url)

In [44]:
image = soup.find("img", src="/assets/logo_nasa_trio_black@2x.png")
print(image)
# featured_image_url = ("https://www.jpl.nasa.gov" + image)
# print(featured_image_url)

<img alt="" class="print_only print_logo" src="/assets/logo_nasa_trio_black@2x.png"/>


In [45]:
# Mars Weather
# Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called mars_weather.
# Example:
# mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'


# visit the mars weather report twitter and scrape the latest tweet
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
time.sleep(1)
mars_weather_html = browser.html
mars_weather_soup = bs(mars_weather_html, 'html.parser')

tweets = mars_weather_soup.find('ol', class_='stream-items')
mars_weather = tweets.find('p', class_="tweet-text").text
print(mars_weather)

InSight sol 157 (2019-05-06) low -100.2ºC (-148.4ºF) high -18.4ºC (-1.2ºF)
winds from the W at 4.1 m/s (9.2 mph) gusting to 14.6 m/s (32.7 mph)
pressure at 7.40 hPapic.twitter.com/R6BliV8xpj


In [46]:
# Mars Facts

# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts 
# about the planet including Diameter, Mass, etc.
# Use Pandas to convert the data to a HTML table string.

# visit space facts and scrap the mars facts table
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)
time.sleep(1)
mars_facts_html = browser.html
mars_facts_soup = bs(mars_facts_html, 'html.parser')

fact_table = mars_facts_soup.find('table', class_='tablepress tablepress-id-mars')
column1 = fact_table.find_all('td', class_='column-1')
column2 = fact_table.find_all('td', class_='column-2')

facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
    })

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_facts

,Facet,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [47]:
# Mars Hemispheres

# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# Save both the image url string for the full resolution hemisphere image, 
# and the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys img_url and title.
# Append the dictionary with the image url string and
# the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [48]:
import time 
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html
soup = bs(html, "html.parser")
mars_hemisphere = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup=bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})

In [49]:
pprint(mars_hemisphere)

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere '}]


In [ ]:
# Step 2 - MongoDB and Flask Application

# Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.
# Start by converting your Jupyter notebook into a Python script called scrape_mars.py with a function called 
# scrape that will execute all of your scraping code from above and return one Python dictionary containing 
# all of the scraped data.

# Next, create a route called /scrape that will import your scrape_mars.py script and call your scrape function.
# Store the return value in Mongo as a Python dictionary.
# Create a root route / that will query your Mongo database and pass the mars data into an HTML template to display the data.
# Create a template HTML file called index.html that will take the mars data dictionary and display all of the 
# data in the appropriate HTML elements. Use the following as a guide for what the final product should look like,
# but feel free to create your own design.


